<a href="https://colab.research.google.com/github/Yanina-Kutovaya/RecSys-retail/blob/main/notebooks/service_test_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
#!unzip awscliv2.zip
#!sudo ./aws/install
#!aws configure

In [2]:
#!git clone https://github.com/Yanina-Kutovaya/RecSys-retail.git
#!pip install -r RecSys-retail/requirements_Colab.txt

In [3]:
%cd RecSys-retail

/content/RecSys-retail


In [6]:
#!dvc pull

In [7]:
import sys
import os

sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), "service"))

import logging
import pandas as pd
from fastapi.testclient import TestClient

from main import app, Model
from src.recsys_retail.models.serialize import load
from src.recsys_retail.data.make_dataset import load_data
from src.recsys_retail.models.train import data_preprocessing_pipeline
from scripts.train_save_model import train_store

In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
logger = logging.getLogger(__name__)

client = TestClient(app)

MODEL = os.getenv("MODEL", default="baseline_v1")

In [10]:
def test_healthcheck():
    response = client.get("/")
    assert response.status_code == 200
    assert response.json()["status"] == "Green"
    logger.info(f'status_code = 200, status = "Green"')

In [11]:
def test_predict():
    data, item_features, user_features = load_data()
    train_dataset_lvl_2 = data_preprocessing_pipeline(
        data, item_features, user_features
    )
    train_store(train_dataset_lvl_2, "baseline_v1")
    Model.classifier = load(MODEL)

    user = {"user_id": 1340}
    response = client.post("/predict?user_id=1340", json=user)
    assert response.status_code == 200
    assert response.json()[0]["user_id"] == 1340

    logger.info(
        f"test_predict single user status_code = 200, response: {response.json()}"
    )

    users = {"user_ids": [1340, 1364]}
    response = client.post("/predict_user_list?batch_id=1", json=users)
    assert response.status_code == 200
    assert response.json()[0]["user_ids"] == [1340, 1364]

    logger.info(
        f"test_predict user_list status_code = 200, response: {response.json()}"
    )

In [12]:
test_healthcheck()
test_predict()

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/2501 [00:00<?, ?it/s]

0:	test: 0.6712357	best: 0.6712357 (0)	total: 131ms	remaining: 2m 10s
50:	test: 0.7896004	best: 0.7902025 (32)	total: 4.37s	remaining: 1m 21s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.7902024935
bestIteration = 32

Shrink model to first 33 iterations.
